## SparseMAX 

let delta^(K-1) = {}

In [116]:
"""Sparsemax activation function.

Pytorch implementation of Sparsemax function from:
-- "From Softmax to Sparsemax: A Sparse Model of Attention and Multi-Label Classification"
-- André F. T. Martins, Ramón Fernandez Astudillo (http://arxiv.org/abs/1602.02068)
"""

from __future__ import division

import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class Sparsemax(nn.Module):
    """Sparsemax function."""

    def __init__(self, dim=None):
        """Initialize sparsemax activation
        
        Args:
            dim (int, optional): The dimension over which to apply the sparsemax function.
        """
        super(Sparsemax, self).__init__()

        self.dim = -1 if dim is None else dim

    def forward(self, input):
        """Forward function.

        Args:
            input (torch.Tensor): Input tensor. First dimension should be the batch size

        Returns:
            torch.Tensor: [batch_size x number_of_logits] Output tensor

        """
        # Sparsemax currently only handles 2-dim tensors,
        # so we reshape to a convenient shape and reshape back after sparsemax
        input = input.transpose(0, self.dim)
        original_size = input.size()
        input = input.reshape(input.size(0), -1)
        input = input.transpose(0, 1)
        dim = 1

        number_of_logits = input.size(dim)

        # Translate input by max for numerical stability
        input = input - torch.max(input, dim=dim, keepdim=True)[0].expand_as(input)

        # Sort input in descending order.
        # (NOTE: Can be replaced with linear time selection method described here:
        # http://stanford.edu/~jduchi/projects/DuchiShSiCh08.html)
        zs = torch.sort(input=input, dim=dim, descending=True)[0]
        range = torch.arange(start=1, end=number_of_logits + 1, step=1, device=device, dtype=input.dtype).view(1, -1)
        range = range.expand_as(zs)

        # Determine sparsity of projection
        bound = 1 + range * zs
        cumulative_sum_zs = torch.cumsum(zs, dim)
        is_gt = torch.gt(bound, cumulative_sum_zs).type(input.type())
        k = torch.max(is_gt * range, dim, keepdim=True)[0]

        # Compute threshold function
        zs_sparse = is_gt * zs

        # Compute taus
        taus = (torch.sum(zs_sparse, dim, keepdim=True) - 1) / k
        taus = taus.expand_as(input)

        # Sparsemax
        self.output = torch.max(torch.zeros_like(input), input - taus)

        # Reshape back to original shape
        output = self.output
        output = output.transpose(0, 1)
        output = output.reshape(original_size)
        output = output.transpose(0, self.dim)

        return output

    def backward(self, grad_output):
        """Backward function."""
        dim = 1

        nonzeros = torch.ne(self.output, 0)
        sum = torch.sum(grad_output * nonzeros, dim=dim) / torch.sum(nonzeros, dim=dim)
        self.grad_input = nonzeros * (grad_output - sum.expand_as(grad_output))

        return self.grad_input

In [119]:
import torch

sparsemax = Sparsemax(dim=1)

logits = torch.randn(2, 5)
print("\nLogits")
print(logits)

sparsemax_probs = sparsemax(logits)
print("\nSparsemax probabilities")
print(sparsemax_probs)


Logits
tensor([[-0.2096,  0.4682, -1.0835,  1.2378, -0.9220],
        [-0.3716,  0.0984, -0.4024, -0.1276, -0.7100]])

Sparsemax probabilities
tensor([[0.0000, 0.1152, 0.0000, 0.8848, 0.0000],
        [0.0792, 0.5492, 0.0484, 0.3232, 0.0000]])


In [128]:

# (BATCH X SEQ)
dtype = torch.float32 
BATCH = 2
SEQ = 5

u = logits

print(u)

tensor([[-0.2096,  0.4682, -1.0835,  1.2378, -0.9220],
        [-0.3716,  0.0984, -0.4024, -0.1276, -0.7100]])


In [129]:
u.shape

torch.Size([2, 5])

In [130]:
input = u
input = input.transpose(0,-1)
original_size = input.size()
print(original_size)
input = input.reshape(input.size(0), -1)
print(input.size())
input = input.transpose(0, 1)
print(input.size())
dim = 1

number_of_logits = input.size(dim)
print(number_of_logits)

torch.Size([5, 2])
torch.Size([5, 2])
torch.Size([2, 5])
5


In [131]:
input = input - torch.max(input, dim=dim, keepdim=True)[0].expand_as(input)
print(input)

tensor([[-1.4474, -0.7696, -2.3213,  0.0000, -2.1598],
        [-0.4700,  0.0000, -0.5008, -0.2260, -0.8084]])


In [132]:
# Sort input in descending order.
# (NOTE: Can be replaced with linear time selection method described here:
# http://stanford.edu/~jduchi/projects/DuchiShSiCh08.html)
zs = torch.sort(input=input, dim=dim, descending=True)[0]
print(zs)
range = torch.arange(start=1, end=number_of_logits + 1, step=1, dtype=input.dtype).view(1, -1)
print(range)
range = range.expand_as(zs)
print(range)

tensor([[ 0.0000, -0.7696, -1.4474, -2.1598, -2.3213],
        [ 0.0000, -0.2260, -0.4700, -0.5008, -0.8084]])
tensor([[1., 2., 3., 4., 5.]])
tensor([[1., 2., 3., 4., 5.],
        [1., 2., 3., 4., 5.]])


In [135]:
# Determine sparsity of projection
bound = 1 + range * zs
print(bound)
cumulative_sum_zs = torch.cumsum(zs, dim)
print(cumulative_sum_zs)
is_gt = torch.gt(bound, cumulative_sum_zs).type(input.type())
k = torch.max(is_gt * range, dim, keepdim=True)[0]

# Compute threshold function
zs_sparse = is_gt * zs

# Compute taus
taus = (torch.sum(zs_sparse, dim, keepdim=True) - 1) / k
taus = taus.expand_as(input)

# Sparsemax
output = torch.max(torch.zeros_like(input), input - taus)

# Reshape back to original shape
output_replication = output
output = output.transpose(0, 1)
output = output.reshape(original_size)
output = output.transpose(0, -1)

tensor([[  1.0000,  -0.5392,  -3.3423,  -7.6391, -10.6063],
        [  1.0000,   0.5480,  -0.4100,  -1.0032,  -3.0419]])
tensor([[ 0.0000, -0.7696, -2.2170, -4.3768, -6.6981],
        [ 0.0000, -0.2260, -0.6960, -1.1968, -2.0052]])


In [134]:
print(output)

tensor([[0.0000, 0.1152, 0.0000, 0.8848, 0.0000],
        [0.0792, 0.5492, 0.0484, 0.3232, 0.0000]])


## Sparse K attention

### The differentiable SparseK Operator

![Sparse Attention](./asset/SparseKimage.png)



In [170]:
import torch

dtype = torch.float32 
BATCH = 1
SEQ = 10
DIM = 2

# (BATCH X SEQ)

u = torch.randn(BATCH, SEQ , dtype=dtype)

print(u)

tensor([[-1.0723, -1.0590,  0.3733,  0.2973, -0.5252, -0.4214,  1.2907, -0.3438,
         -0.9316, -1.6184]])



![W and U ](./asset/uandw.png)


In [ ]:
import torch

dtype = torch.float32 
BATCH = 1
SEQ = 10
DIM = 2
# ...  setup ...
u = torch.randn(BATCH, SEQ , dtype=dtype)
z = u
zminus = z - 1


# Merge zminus and z to find beta candidates 
merged_tensor = torch.cat((z, zminus), dim=1)
merged_sorted_tensor, _ = torch.sort(merged_tensor, descending=True)
# merged_sorted_tensor are candidate for beta (referering  to the 2m distinct pairs )
merged_sorted_tensor = merged_sorted_tensor[0]

In [248]:
print("Z input")
print(z)

print("Beta Candidate List")
print(merged_sorted_tensor)

Z input
tensor([[ 0.7709, -0.4365,  0.9997, -0.8282, -3.0413,  0.8198, -0.0842, -1.0837,
          0.3916,  0.3448]])
Beta Candidate List
tensor([ 9.9969e-01,  8.1984e-01,  7.7087e-01,  3.9159e-01,  3.4476e-01,
        -3.1495e-04, -8.4162e-02, -1.8016e-01, -2.2913e-01, -4.3646e-01,
        -6.0841e-01, -6.5524e-01, -8.2824e-01, -1.0837e+00, -1.0842e+00,
        -1.4365e+00, -1.8282e+00, -2.0837e+00, -3.0413e+00, -4.0413e+00])


In [249]:
pairs = []  # will store (U_count, W_count) as *counts*
for beta in merged_sorted_tensor:
    b = beta.item()

    # equation 24
    # masks over sorted z
    u_ge_mask = (z >= b + 1)      # will be True for indices [0 .. U-1]
    w_gt_mask = (z > b)           # will be True for indices [0 .. W-1]

    U = int(u_ge_mask.sum().item())   # <-- counts, not indices
    W = int(w_gt_mask.sum().item())   # <-- counts, not indices
    if W <= U:                        # need at least one fractional slot
        continue

    pairs.append((U, W))

In [250]:
print("U,W pairs from all beta candidates")
print(pairs)

U,W pairs from all beta candidates
[(0, 1), (0, 2), (0, 3), (0, 4), (1, 5), (1, 5), (2, 6), (3, 6), (3, 6), (4, 7), (5, 7), (5, 7), (5, 8), (5, 9), (6, 9), (8, 9)]



![Algorithm 1](./asset/algorithm1.png)

In [357]:
# Sort z (descending) for the same batch
z, _ = torch.sort(z, descending=True)
z = z[0]                                  # 1D scores (length m)
m = z.numel()

In [358]:
# pre comute the cumulative sum
z_cum = torch.cumsum(z, dim=0)            # S[t] = sum_{i=0..t} z[i]
z_pad = torch.cat([z, z.new_tensor([-float('inf')])])  # z[m] = -inf for right-guard

##### within iteration

In [359]:
U = 0 
W = 1
seg_sum = z_cum[W-1] - (z_cum[U-1] if U > 0 else z_cum.new_zeros(()))
tau_cand = (seg_sum + (U - k)) / (W - U)

![Condition](./asset/condition.png)

In [360]:
# interval checks (use sentinels; interpret counts -> 0-based indices)
left_cond  = (z[W-1] > tau_cand) if W >= 1 else True
right_cond = (tau_cand >= z_pad[W])  # z_pad[W] is safe for W==m
up_cond    = (z[U-1] >= tau_cand + 1) if U >= 1 else True
down_cond  = ((tau_cand + 1) > z_pad[U])  # z_pad[U] safe for U==m

In [361]:
left_cond , right_cond , up_cond , down_cond

(tensor(True), tensor(False), True, tensor(False))

then break if all true

###

In [362]:
# scan in the paper's order: descending in beta (pairs already collected that way)
tau = None
k = 3.5
for U, W in pairs:
    # sums over i in [U .. W-1] using zero-based inclusive prefix sums:
    # sum_{U..W-1} z = z_cum[W-1] - (z_cum[U-1] if U>0 else 0)
    seg_sum = z_cum[W-1] - (z_cum[U-1] if U > 0 else z_cum.new_zeros(()))
    tau_cand = (seg_sum + (U - k)) / (W - U)

    # interval checks (use sentinels; interpret counts -> 0-based indices)
    left_ok  = (z[W-1] > tau_cand) if W >= 1 else True
    right_ok = (tau_cand >= z_pad[W])  # z_pad[W] is safe for W==m
    up_ok    = (z[U-1] >= tau_cand + 1) if U >= 1 else True
    down_ok  = ((tau_cand + 1) > z_pad[U])  # z_pad[U] safe for U==m

    if left_ok and right_ok and up_ok and down_ok:
        tau = tau_cand
        break
    
p = torch.clamp(z - tau, 0, 1)
print("sum p =", float(p.sum()))

TypeError: unsupported operand type(s) for -: 'Tensor' and 'NoneType'

In [270]:
p

tensor([1.0000, 0.8631, 0.8141, 0.4348, 0.3880, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000])

### Assuming p is correct, sum of p == k

In [266]:
# (BATCH X SEQ X DIM)
# output of q, k ,v single head

q = torch.randn(BATCH, 1, SEQ, DIM, dtype=dtype)
key = torch.randn(BATCH, 1, SEQ, DIM, dtype=dtype)
v = torch.randn(BATCH, 1, SEQ, DIM, dtype=dtype)

x = torch.randn(BATCH, SEQ , DIM)

![Selection_matrix](./asset/selecton_matrix.png)

In [267]:
u, k

(tensor([[ 0.7709, -0.4365,  0.9997, -0.8282, -3.0413,  0.8198, -0.0842, -1.0837,
           0.3916,  0.3448]]),
 3.5)

In [271]:
def topk_mask(u, k):
    """
    u: (B, T) importance scores up to current position i
    k: number of key-value pairs to select
    returns: m_topk: binary mask (B, T)
    """
    B, T = u.shape
    m_topk = torch.zeros_like(u, dtype=torch.float32)

    # select top-k indices
    topk_vals, topk_idx = torch.topk(u, min(k, T), dim=1)
    m_topk[torch.arange(B).unsqueeze(1), topk_idx] = 1.0
    return m_topk

def mask_select_diag( msparsek, mtopk):
    """
    Args:
        msparsek: Tensor of shape (T,) or (1, T) with soft scores
        mtopk: Tensor of shape (T,) or (1, T) with 0/1 hard selection

    Returns:
        Tensor of shape (T, T): Diagonal matrix with masked scores
    """
    # Ensure the vectors are 1D
    msparsek = msparsek.flatten()
    mtopk = mtopk.flatten()
    
    # Create diagonal matrix from msparsek
    diag_matrix = torch.diag(msparsek)
    
    # Apply row mask: keep only rows where mtopk == 1
    mask = mtopk.unsqueeze(1).expand_as(diag_matrix)  # shape (T, T)
    masked_matrix = diag_matrix * mask.float()
    
    return masked_matrix


tkmasking = topk_mask(u, int(k))
print("TOP K mask")
print(tkmasking)
selection_mat = mask_select_diag(p, tkmasking)
print("Soft selection")
print(selection_mat.shape)

TOP K mask
tensor([[1., 0., 1., 0., 0., 1., 0., 0., 0., 0.]])
Soft selection
torch.Size([10, 10])


In [272]:
k_hat = selection_mat @ key
v_hat = selection_mat @ v

k_hat.shape , v_hat.shape

(torch.Size([1, 1, 10, 2]), torch.Size([1, 1, 10, 2]))

In [287]:
import torch.nn.functional as F
p = F.softmax(q @ k_hat.transpose(-2,-1))
p.shape


C:\Users\teeds\AppData\Local\Temp\ipykernel_1912\3235830935.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  p = F.softmax(q @ k_hat.transpose(-2,-1))


torch.Size([1, 1, 10, 10])

In [288]:
p.shape , v_hat.shape

(torch.Size([1, 1, 10, 10]), torch.Size([1, 1, 10, 2]))

In [289]:
p.transpose(-2,-1).shape

torch.Size([1, 1, 10, 10])

In [292]:
o = p.transpose(-2,-1) @ v_hat
o.shape

torch.Size([1, 1, 10, 2])

In [ ]:
def sparsek_1d(z, k):


    # Merge zminus and z to find beta candidates 
    merged_tensor = torch.cat((z, z - 1), dim=1)
    merged_sorted_tensor, _ = torch.sort(merged_tensor, descending=True)
    # merged_sorted_tensor are candidate for beta (referering  to the 2m distinct pairs )
    merged_sorted_tensor = merged_sorted_tensor[0]

    # Sort z (descending) for the same batch
    z, _ = torch.sort(z, descending=True)
    z = z[0]                                  # 1D scores (length m)

    # pre comute the cumulative sum
    z_cum = torch.cumsum(z, dim=0)            # S[t] = sum_{i=0..t} z[i]
    z_pad = torch.cat([z, z.new_tensor([-float('inf')])])  # z[m] = -inf for right-guard


    pairs = []  # will store (U_count, W_count) as *counts*
    for beta in merged_sorted_tensor:
        b = beta.item()

        # equation 24
        # masks over sorted z
        u_ge_mask = (z >= b + 1)      # will be True for indices [0 .. U-1]
        w_gt_mask = (z > b)           # will be True for indices [0 .. W-1]

        U = int(u_ge_mask.sum().item())   # <-- counts, not indices
        W = int(w_gt_mask.sum().item())   # <-- counts, not indices
        if W <= U:                        # need at least one fractional slot
            continue

        pairs.append((U, W))

    print("U,W pairs from all beta candidates")
    print(pairs)

    
    # scan in the paper's order: descending in beta (pairs already collected that way)
    tau = None

    for U, W in pairs:
        # sums over i in [U .. W-1] using zero-based inclusive prefix sums:
        # sum_{U..W-1} z = z_cum[W-1] - (z_cum[U-1] if U>0 else 0)
        seg_sum = z_cum[W-1] - (z_cum[U-1] if U > 0 else z_cum.new_zeros(()))
        tau_cand = (seg_sum + (U - k)) / (W - U)

        # interval checks (use sentinels; interpret counts -> 0-based indices)
        left_ok  = (z[W-1] > tau_cand) if W >= 1 else True
        right_ok = (tau_cand >= z_pad[W])  # z_pad[W] is safe for W==m
        up_ok    = (z[U-1] >= tau_cand + 1) if U >= 1 else True
        down_ok  = ((tau_cand + 1) > z_pad[U])  # z_pad[U] safe for U==m

        if left_ok and right_ok and up_ok and down_ok:
            tau = tau_cand
            break
        
    p = torch.clamp(z - tau, 0, 1)
    print("sum p =", float(p.sum()))
    return p

In [307]:
BATCH = 1
u = torch.randn(BATCH, SEQ , dtype=dtype)

p = sparsek_nd(u, 3)
print(p)
print("sum p =", float(p.sum()))

tensor([[1.0000, 0.6552, 0.5831, 0.5766, 0.1068, 0.0784, 0.0000, 0.0000, 0.0000,
         0.0000]])
sum p = 2.999999523162842


In [325]:
BATCH = 3
SEQ = 10
u = torch.randn(BATCH, SEQ , dtype=dtype)

z = u 


z.shape

torch.Size([3, 10])

In [326]:
# Merge zminus and z to find beta candidates 
merged_tensor = torch.cat((z, z - 1), dim=1)
print(merged_tensor.shape)
merged_sorted_tensor, _ = torch.sort(merged_tensor, descending=True , dim=1)
print(merged_sorted_tensor.shape)
# merged_sorted_tensor are candidate for beta (referering  to the 2m distinct pairs )
print(merged_sorted_tensor[0])


torch.Size([3, 20])
torch.Size([3, 20])
tensor([ 2.0092,  1.8055,  1.0092,  0.8055,  0.7135,  0.6731,  0.3174, -0.2865,
        -0.3269, -0.6826, -0.8289, -0.8739, -1.2162, -1.4271, -1.4842, -1.8289,
        -1.8739, -2.2162, -2.4271, -2.4842])


In [327]:
z[0]

tensor([-1.2162, -1.4271,  0.6731, -0.8289,  1.8055, -0.8739,  0.3174, -1.4842,
         0.7135,  2.0092])

In [328]:
# Sort z descending for each batch
z_sorted, _ = torch.sort(z, descending=True, dim=1)  # (B, m)

# Cumulative sum for each batch
z_cum = torch.cumsum(z_sorted, dim=1)  # (B, m)

In [329]:
z_sorted[0] 

tensor([ 2.0092,  1.8055,  0.7135,  0.6731,  0.3174, -0.8289, -0.8739, -1.2162,
        -1.4271, -1.4842])

same number different order

In [330]:
z_cum.shape , z_cum[0]

(torch.Size([3, 10]),
 tensor([ 2.0092,  3.8147,  4.5283,  5.2013,  5.5187,  4.6898,  3.8159,  2.5997,
          1.1726, -0.3116]))

In [331]:
# Right-guard padding
z_pad = torch.cat([z_sorted, z.new_full((BATCH, 1), -float('inf'))], dim=1)  # (B, m+1)
z_pad.shape

torch.Size([3, 11])

In [ ]:
p_out = []
for b in range(BATCH):
    pairs = []
    print(merged_sorted_tensor.shape)
    for beta in merged_sorted_tensor[b]:
        bval = beta.item()
        u_ge_mask = (z_sorted[b] >= bval + 1)
        w_gt_mask = (z_sorted[b] > bval)
        U = int(u_ge_mask.sum().item())
        W = int(w_gt_mask.sum().item())
        if W <= U:
            continue
        pairs.append((U, W))
    tau = None
    print(pairs)
    for U, W in pairs:
        print(U, W)
        seg_sum = z_cum[b, W-1] - (z_cum[b, U-1] if U > 0 else 0)
        tau_cand = (seg_sum + (U - k)) / (W - U)

        left_ok  = (z_sorted[b, W-1] > tau_cand) if W >= 1 else True
        right_ok = (tau_cand >= z_pad[b, W])
        up_ok    = (z_sorted[b, U-1] >= tau_cand + 1) if U >= 1 else True
        down_ok  = ((tau_cand + 1) > z_pad[b, U])

        if left_ok and right_ok and up_ok and down_ok:
            tau = tau_cand
            break
    print(tau)
    p_b = torch.clamp(z_sorted[b] - tau, 0, 1)
    p_out.append(p_b)

torch.Size([3, 20])
torch.Size([10])
torch.Size([10])
torch.Size([10])
torch.Size([10])
torch.Size([10])
torch.Size([10])
torch.Size([10])
torch.Size([10])
torch.Size([10])
torch.Size([10])
torch.Size([10])
torch.Size([10])
torch.Size([10])
torch.Size([10])
torch.Size([10])
torch.Size([10])
torch.Size([10])
torch.Size([10])
torch.Size([10])
torch.Size([10])
[(0, 1), (1, 2), (2, 3), (2, 4), (3, 5), (4, 5), (5, 6), (5, 7), (5, 8), (5, 9), (6, 10), (7, 10), (7, 10), (9, 10)]
0 1
1 2
2 3
2 4
3 5
4 5
5 6
5 7
5 8
5 9
6 10
7 10
7 10
9 10
None


TypeError: unsupported operand type(s) for -: 'Tensor' and 'NoneType'

Need a fallback 

In [348]:
len(list_of_paris) , len(list_of_paris[0]), len(list_of_paris[0][0])

(3, 14, 2)

In [ ]:
list_tau = []
for b in range(BATCH):
    pairs = list_of_paris[b]
    tau = None
    for U, W in pairs:
        seg_sum = z_cum[b, W-1] - (z_cum[b, U-1] if U > 0 else 0)
        tau_cand = (seg_sum + (U - k)) / (W - U)

        left_ok  = (z_sorted[b, W-1] > tau_cand) if W >= 1 else True
        right_ok = (tau_cand >= z_pad[b, W])
        up_ok    = (z_sorted[b, U-1] >= tau_cand + 1) if U >= 1 else True
        down_ok  = ((tau_cand + 1) > z_pad[b, U])

        if left_ok and right_ok and up_ok and down_ok:
            tau = tau_cand
            list_tau.append(tau)
            break


[(0, 1), (1, 2), (2, 3), (2, 4), (3, 5), (4, 5), (5, 6), (5, 7), (5, 8), (5, 9), (6, 10), (7, 10), (7, 10), (9, 10)]
0 1
1 2
2 3
2 4
3 5
4 5
5 6
5 7
5 8
5 9
6 10
7 10
7 10
9 10
[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (1, 6), (2, 6), (3, 6), (3, 6), (3, 7), (4, 8), (4, 8), (5, 9), (6, 9), (6, 9), (7, 10), (7, 10), (9, 10), (9, 10)]
0 1
0 2
0 3
0 4
0 5
1 6
[(0, 1), (0, 2), (0, 3), (1, 4), (1, 4), (1, 5), (1, 6), (1, 7), (2, 8), (2, 8), (2, 9), (2, 10), (3, 10), (4, 10), (6, 10), (7, 10), (8, 10), (9, 10), (9, 10)]
0 1
0 2
0 3
1 4
1 4
1 5
1 6
1 7
2 8
2 8
2 9
2 10
3 10
4 10
6 10
7 10
8 10
9 10
9 10


In [347]:
list_tau

[tensor(0.1836)]

In [ ]:





def learnable_selection(input, w_score , epsilon):
    zs = torch.sort(input=input, dim=dim, descending=True)[0]
    range = torch.arange(start=1, end=number_of_logits + 1, step=1, device=device, dtype=input.dtype).view(1, -1)
    range = range.expand_as(zs)

    # Determine sparsity of projectio
    u = zs  @ w_score + range * epsilon

    return u 

m = input.size(1)

dim = 1
cumulative_sum_zs = torch.cumsum(zs, dim)


